# Imports

In [1]:
import sys
import os

# In Jupyter notebooks, __file__ is not defined. Use workaround:
if "__file__" in globals():
    base_dir = os.path.dirname(__file__)

else:
    base_dir = os.getcwd()
    sys.path.insert(0, os.path.abspath(os.path.join(base_dir, '..', '..')))


from app.agents.real_estate_agent import real_state_agent
from app.models.user_models import UserInput
from pydantic_ai.messages import ToolCallPart
import duckdb

# Functions

# Agent testing

In [2]:
connection = duckdb.connect("../../db/db.duckdb")

## Real estate agent

## Free tests

In [9]:
params = {"bairro":"boqueirao","cidade":"Curitiba","n_quartos":2,"preco_max":500000}
n_quartos = params.get('n_quartos')
preco_max = params.get('preco_max')
cidade = params.get('cidade')
bairro = params.get('bairro')

In [10]:
base_query = "SELECT property_id, preco, tamanho, cidade, bairro, rua, n_quartos, n_banheiros, n_garagem FROM properties"
filters = []
params = {}

if preco_max is not None:
    filters.append(f"preco <= {preco_max}")
    params["preco_max"] = preco_max
if n_quartos is not None:
    filters.append(f"n_quartos = {n_quartos}")
    params["n_quartos"] = n_quartos
if cidade is not None:
    filters.append(f"cidade ILIKE '%{cidade.lower()}%'")
    params["cidade"] = f"%{cidade.lower()}%"
    if bairro is not None:
        filters.append(f"bairro ILIKE '%{bairro.lower()}%'")
        params["bairro"] = f"%{bairro.lower()}%"

if filters:
    base_query += " WHERE " + " AND ".join(filters)

df = connection.execute(base_query).fetch_df()

In [11]:
df

,property_id,preco,tamanho,cidade,bairro,rua,n_quartos,n_banheiros,n_garagem
0,1190f6a63fa2754842367bc615f8ac1c,385000.0,70.0,curitiba,alto boqueirao,Rua Irmã Maria Francisca Michel,2,2,1
1,7c45661eaff5997da3c0359a1a037d83,120000.0,40.0,curitiba,alto boqueirao,"Travessa Célia Cardozo Dos Santos, 137",2,2,3
2,a1d999dfa8484de37ee579d014d56556,392000.0,58.0,curitiba,boqueirao,Rua Zonardy Ribas,2,2,1


In [3]:
user_input = "Olá, eu gostaria de um imóvel. Até 500mil, eu moro em Curitiba, gostaria de algo no bairro Portão, 2 quartos"

tool_names = []
custom_tools = ["search_properties", "get_property_slots", "book_property_slot", "cancel_property_slot"]
async with real_state_agent.iter(user_input, deps=UserInput(connection=connection, user_name="Alex")) as agent_run:
    async for node in agent_run:
        if real_state_agent.is_call_tools_node(node):
            for tool_call in node.model_response.parts:
                if isinstance(tool_call, ToolCallPart) :
                    tool_names.append(tool_call.tool_name)

In [4]:
agent_run

<AgentRun result=FinalResult(output=RealStateAgentOutput(response='Olá Alex, infelizmente não encontrei nenhuma propriedade que atenda aos seus critérios no bairro Portão em Curitiba. Pode tentar novamente com filtros menos específicos? Ou se estiver procurando por uma propriedade específica usando quase todos os parâmetros, revise os valores utilizados. Lembre-se de que os nomes de ruas podem ter variações lexicais, então tente usar um valor diferente para esse parâmetro.', properties='No properties found for the given filters. Please try again with less specific filters. But if you are looking for a specific property using almost all parameters, review parameters values. IMPORTANT: If you are ussing the `rua` parameter to find a specific property, review the value used. Rua names tend to have lexical variations, so try ussing a different value for this parameter.', slots=None), tool_name='final_result', tool_call_id='h1ze91bge') usage=Usage(requests=2, request_tokens=3779, response_t

## Tests

### 1

In [ ]:
user_input = "Hi there!"

tool_names = []
custom_tools = ["search_properties", "get_property_slots", "book_property_slot", "cancel_property_slot"]
async with real_state_agent.iter(user_input, deps=UserInput(connection=connection, user_name="Alex")) as agent_run:
    async for node in agent_run:
        if real_state_agent.is_call_tools_node(node):
            for tool_call in node.model_response.parts:
                if isinstance(tool_call, ToolCallPart) :
                    tool_names.append(tool_call.tool_name)

not any(tool in tool_names for tool in custom_tools)

False

### 2

In [4]:
user_input = "Quais os horários para visitar o imóvel?"

tool_names = []
async with real_state_agent.iter(user_input, deps=UserInput(connection=connection, user_name="Alex")) as agent_run:
    async for node in agent_run:
        if real_state_agent.is_call_tools_node(node):
            for tool_call in node.model_response.parts:
                if isinstance(tool_call, ToolCallPart) :
                    tool_names.append(tool_call.tool_name)

### 3

In [ ]:
user_input = "Gostaria de agendar uma visita para o imóvel 'abcfoo42' no dia 2024-12-25 as 10 da manhã."
#user_input = "Quero agendar uma visita para o imovel de id '32ih7c' as 10 da manhã."

tool_names = []
async with real_state_agent.iter(user_input, deps=UserInput(connection=connection, user_name="Alex")) as agent_run:
    async for node in agent_run:
        if real_state_agent.is_call_tools_node(node):
            for tool_call in node.model_response.parts:
                if isinstance(tool_call, ToolCallPart):
                    tool_names.append(tool_call)